## Классификация отрывков текста

Программа, которая по отрывку текста говорит, принадлежит ли он перу Льва Толстого или Ильи Ильфа и Евгения Петрова.

#### Подготовка данных

Было скачано 4MB книг Ильи Ильфа и Евгения Петрова и 6MB книг Льва Толстого. Далее все тексты были разбиты на отрывки по 100 слов в каждом. Понятно, что чем больше слов в отрывке, тем проще определить автора. Были попробованы модели, обученные на отрывках различной длины. Оптимальным с точки зрения качества классификации и здравого смысла значением длины отрывка оказалось 100 слов.

Также все данные были поделены на обучающую, валидационную и тестовую выборки в соотношении 70/15/15. Отдельно валидационная и тестовая выборки необходимы для того, чтобы не переобучаться на валидационную выборку при подборе гиперпараметров модели, и оценивать итоговое качество модели по тестовой выборке.

#### Векторные представления текстовых данных

Для того, чтобы обучать модели, необходимо было представить текстовые данные в числовом виде. Для этого использовалась модель gensim.models.doc2vec, переводящая текстовые документы в вектора. Модель обучалась на двух текстах - книги Ильфа и Петрова и книги Толстого.

#### Метрика качества

Так как хотелось предсказывать вероятность принадлежности отрывка к одному из классов, то использовалась метрика AUC ROC.

#### Обучение моделей классификации

Было попробовано три модели: градиентный бустинг XGBoost, логистическая регрессия и метод опорных векторов.

Градиентный бустинг показал качество **0.87444** на тестовых данных. Логистическая регрессия: **0.88488**. SVM: **0.88509**. Три решения были скомбинированы ансамблем, итоговое качество на тестовых данных получилось **0.88686**.

In [1]:
import numpy as np
import pymorphy2
from time import time

from gensim.models import doc2vec
from sklearn.model_selection import train_test_split, ParameterGrid
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

### Подготовка данных

In [2]:
file_names_Ilf_Petrov = ['Data/Ilf_Petrov_1001_den_ili_novaya_Shacherezada.txt',
                         'Data/Ilf_Petrov_Dvenadcat_stulev.txt',
                         'Data/Ilf_Petrov_Neobyknovennye_istorii_iz_zhizni_goroda_kolokolamska.txt',
                         'Data/Ilf_Petrov_Odnoetazhnaya_America.txt',
                         'Data/Ilf_Petrov_Svetlaya_lichnost.txt',
                         'Data/Ilf_Petrov_Zolotoj_telenok.txt']
file_names_Tolstoy = ['Data/Tolstoy_Anna_Karenina.txt',
                      'Data/Tolstoy_Vojna_i_mir_Tom_1.txt',
                      'Data/Tolstoy_Voskresenie.txt']

In [3]:
def normalize_text(text):
    """
    Transforms words from text to normal form
    
    Parameters
    ----------
    text (string): text
    
    Returns
    -------
    string: text (list of words) with words in normal form
    """
    
    words = ''.join(char for char in text.lower() if char.isalpha() or char == ' ').split()
    morph = pymorphy2.MorphAnalyzer()
    return [morph.parse(word)[0].normal_form for word in words]

In [4]:
def read_texts(file_names, batch_length, display=False):
    """
    Reads texts from files, normalizes them and returns texts in batches
    
    Parameters
    ----------
    file_names: list of strings
    batch_length: number of words in a batch
    display (boolean): whether to display processed information or not
    
    Returns
    -------
    texts: numpy.ndarray
    """
    
    start_time = time()
    texts = []
    for j, file_name in enumerate(file_names):
        curr_text = [] # current batch
        n_words = 0 # count number of words in a batch
        
        file = open(file_name, 'r')
        for line in file:
            norm_line = normalize_text(line)
            
            while n_words + len(norm_line) > batch_length:
                rem_len = batch_length - n_words # remaining length of the batch
                curr_text += norm_line[:rem_len]
                texts.append(curr_text)
                
                norm_line = norm_line[rem_len:]
                curr_text = []
                n_words = 0
            
            n_words += len(norm_line)
            curr_text += norm_line

        if display:
            print('File #{} has been processed (time passed: {})'.format(j, time() - start_time))

    return np.array(texts)

In [5]:
texts_Ilf_Petrov = read_texts(file_names_Ilf_Petrov, batch_length=100, display=True)

File #0 has been processed (time passed: 37.835206747055054)
File #1 has been processed (time passed: 558.6166124343872)
File #2 has been processed (time passed: 625.0489070415497)
File #3 has been processed (time passed: 1036.5923702716827)
File #4 has been processed (time passed: 1109.520709991455)
File #5 has been processed (time passed: 1560.5091240406036)


In [6]:
texts_Tolstoy = read_texts(file_names_Tolstoy, batch_length=100, display=True)

File #0 has been processed (time passed: 637.1563684940338)
File #1 has been processed (time passed: 1124.6750407218933)
File #2 has been processed (time passed: 1831.6201725006104)


In [8]:
data = np.vstack((texts_Ilf_Petrov, texts_Tolstoy))
target = np.array([0] * len(texts_Ilf_Petrov) + [1] * len(texts_Tolstoy))

In [9]:
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.3, random_state=17)

In [10]:
data_valid, data_test, target_valid, target_test = train_test_split(data_test, target_test,
                                                                    test_size=0.5, random_state=17)

In [11]:
len(data), len(data_train), len(data_valid), len(data_test)

(8298, 5808, 1245, 1245)

Получили 8298 объектов, из них: 5808 объектов для обучения и 1245 для валидации и 1245 для теста.

### Doc2Vec

In [12]:
def preprocess_data(documents):
    """
    Preprocess data (already normalized) before it can be used in gensim.models.doc2vec.Doc2Vec
    
    Parameters
    ----------
        documents: list of texts (where text is a list of words)
    
    Returns
    -------
        list of tagged documents
    """
    
    tagged_documents = []
    for i, document in enumerate(documents):
        tagged_documents.append(doc2vec.TaggedDocument(document, [i]))
    return tagged_documents

In [13]:
d2v_data = [data_train[target_train == 0].flatten(),
            data_train[target_train == 1].flatten()]
tagged_data = preprocess_data(d2v_data)

In [14]:
vec_size = 100
d2v_model = doc2vec.Doc2Vec(tagged_data, size=vec_size, window=8, min_count=3, workers=4)

#### Преобразуем все выборки в вектора

In [15]:
data_train_d2v = np.empty((0, vec_size))
for text in data_train:
    data_train_d2v = np.vstack((data_train_d2v, d2v_model.infer_vector(text)))

data_valid_d2v = np.empty((0, vec_size))
for text in data_valid:
    data_valid_d2v = np.vstack((data_valid_d2v, d2v_model.infer_vector(text)))

data_test_d2v = np.empty((0, vec_size))
for text in data_test:
    data_test_d2v = np.vstack((data_test_d2v, d2v_model.infer_vector(text)))

In [16]:
def clf_new_document(document, d2v_model, clf_model):
    """
    Classifies new text document
    
    Parameters
    ----------
    document: numpy.ndarray of words
    d2v_model: gensim.models.doc2vec.Doc2Vec
    clf_model: sklearn trained model with predict_proba method
    
    Returns
    ----------
    float: probability of class 1
    """
    
    # normalize document text
    norm_document = normalize_text(' '.join(document))[0]
    
    # get vector for new document with doc2vec model
    vec = d2v_model.infer_vector(norm_document.split())
    
    # predict class with trained classifier model 
    return clf_model.predict_proba(vec.reshape(1, -1))[0][1]

### XGBoost

In [17]:
param_grid = {'max_depth': [4, 5],
              'n_estimators': [80, 90],
              'min_child_weight': [2, 3],
              'gamma': [0.001, 0.01],
              'reg_lambda': [2, 3],
              'reg_alpha': [1, 2],
              'learning_rate': [0.1, 0.2],
              'nthread': [10],
              'seed': [17]}
start_time = time()
scores = []
for i, params in enumerate(ParameterGrid(param_grid)):
    clf_xgb = xgb.XGBClassifier(**params)
    clf_xgb.fit(data_train_d2v, target_train)
    
    pred_xgb = []
    for vec in data_valid_d2v:
        pred_xgb.append(clf_xgb.predict_proba(vec.reshape(1, -1))[0][1])
    
    scores.append(roc_auc_score(target_valid, pred_xgb))

In [18]:
best_param = ParameterGrid(param_grid)[np.argmax(np.array(scores))]
best_param, max(scores)

({'gamma': 0.001,
  'learning_rate': 0.2,
  'max_depth': 4,
  'min_child_weight': 3,
  'n_estimators': 90,
  'nthread': 10,
  'reg_alpha': 1,
  'reg_lambda': 3,
  'seed': 17},
 0.87263150374782927)

In [19]:
clf_xgb_best = xgb.XGBClassifier(**best_param)
clf_xgb_best.fit(data_train_d2v, target_train)

pred_xgb = []
for vec in data_test_d2v:
    pred_xgb.append(clf_xgb_best.predict_proba(vec.reshape(1, -1))[0][1])

In [20]:
roc_auc_score(target_test, pred_xgb)

0.87443706123890685

### Логистическая регрессия

In [21]:
start_time = time()
scores = []
Cs = np.linspace(1000, 2000, 100)
for C in Cs:
    clf_log_reg = LogisticRegression(C=C, random_state=17)
    clf_log_reg.fit(data_train_d2v, target_train)
    
    pred_log_reg = []
    for vec in data_valid_d2v:
        pred_log_reg.append(clf_log_reg.predict_proba(vec.reshape(1, -1))[0][1])
    
    scores.append(roc_auc_score(target_valid, pred_log_reg))

In [22]:
best_C_log_reg = Cs[np.argmax(np.array(scores))]
best_C_log_reg, max(scores)

(1030.3030303030303, 0.89127997713934026)

In [23]:
clf_log_reg_best = LogisticRegression(C=best_C_log_reg, random_state=17)
clf_log_reg_best.fit(data_train_d2v, target_train)

pred_log_reg = []
for vec in data_test_d2v:
    pred_log_reg.append(clf_log_reg_best.predict_proba(vec.reshape(1, -1))[0][1])

In [24]:
roc_auc_score(target_test, pred_log_reg)

0.88488178286017038

### SVM

In [27]:
start_time = time()
scores = []
Cs = np.linspace(5000, 10000, 20)
for C in Cs:
    clf_svm = SVC(C=C, probability=True, random_state=17)
    clf_svm.fit(data_train_d2v, target_train)
    
    pred_svm = []
    for vec in data_valid_d2v:
        pred_svm.append(clf_svm.predict_proba(vec.reshape(1, -1))[0][1])
    
    scores.append(roc_auc_score(target_valid, pred_svm))

In [28]:
best_C_svm = Cs[np.argmax(np.array(scores))]
best_C_svm, max(scores)

(5526.3157894736842, 0.89231172927703173)

In [29]:
clf_svm_best = SVC(C=best_C_svm, probability=True, random_state=17)
clf_svm_best.fit(data_train_d2v, target_train)

pred_svm = []
for vec in data_test_d2v:
    pred_svm.append(clf_svm.predict_proba(vec.reshape(1, -1))[0][1])

In [30]:
roc_auc_score(target_test, pred_svm)

0.88508598613625344

### Models ensemble

In [31]:
clf_xgb_best = xgb.XGBClassifier(**best_param)
clf_log_reg_best = LogisticRegression(C=best_C_log_reg, random_state=17)
clf_svm_best = SVC(C=best_C_svm, probability=True, random_state=17)

In [36]:
clf_ens_best = VotingClassifier(estimators=[('xgb', clf_xgb_best),
                                            ('lr', clf_log_reg_best),
                                            ('svm', clf_svm_best)], voting='soft')
clf_ens_best.fit(data_train_d2v, target_train)

pred_ens = []
for vec in data_test_d2v:
    pred_ens.append(clf_ens_best.predict_proba(vec.reshape(1, -1))[0][1])

In [37]:
roc_auc_score(target_test, pred_ens)

0.88686310653892009